In [ ]:
 pip install langchain_openai langchain_community langchain pymysql chromadb pyngrok flask -q

In [37]:
 import os
 os.environ["OPENAI_API_KEY"] = "sk-proj-TzuO4YHFVwaw8sTLcupST3BlbkFJaHHZh88uu0IYhFhI3htF"

 db_user = "hackathon"
 db_password = "hackathon"
 db_host = "panosgio.org"
 db_name = "hackathon"
 from langchain_community.utilities.sql_database import SQLDatabase
 db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")


In [ ]:
# Import necessary libraries
from flask import Flask, request, jsonify, render_template, send_from_directory, make_response
from pyngrok import ngrok, conf
import requests
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# Initialize Flask app

conf.get_default().auth_token = "2fP5qkG9bfrHhQKYeqbX3VPY7bT_6Rda87eSFwgcpX7vcmBr"

app = Flask(__name__)

@app.route('/')
def index():
    # Load HTML content
    html_url = 'https://panosgio.org/hackathon/index.html'
    html_response = requests.get(html_url)
    html_content = html_response.text

    # Load CSS content
    css_url = 'https://panosgio.org/hackathon/ui.css'
    css_response = requests.get(css_url)
    css_content = css_response.text

    # Load JavaScript content
    js_url = 'https://panosgio.org/hackathon/ui.js'
    js_response = requests.get(js_url)
    js_content = js_response.text

    # Embed CSS and JS into HTML
    html_content = html_content.replace('<link rel="stylesheet" href="ui.css">', '<style>' + css_content + '</style>')
    html_content = html_content.replace('<script src="ui.js"></script>', '<script>' + js_content + '</script>')

    # Return the modified HTML content
    return make_response(html_content)

# Define the endpoint
@app.route('/run_query', methods=['POST'])
def run_query():

    question = request.json.get('question')


    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    generate_query = create_sql_query_chain(llm, db, k=1000)
    query = generate_query.invoke({"question": question})
    # print(query)
    # answer_prompt = PromptTemplate.from_template("write it to sql format. : SQL Query: {query} Answer: ")
    # rephrase_answer = answer_prompt | llm | StrOutputParser()
    # print(rephrase_answer)
    # # execute_query = QuerySQLDataBaseTool(db=db)
    # chain = (
    #     RunnablePassthrough.assign(query=generate_query).assign(
    #         result=itemgetter("query") | execute_query
    #     )
    #     | rephrase_answer
    # )

    # query2 = chain.invoke({"question": question})
    print(query)
    execute_query = QuerySQLDataBaseTool(db=db)
    execute_query.invoke(query)


    answer_prompt = PromptTemplate.from_template(
        """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer: """
    )

    rephrase_answer = answer_prompt | llm | StrOutputParser()

    chain = (
        RunnablePassthrough.assign(query=generate_query).assign(
            result=itemgetter("query") | execute_query
        )
        | rephrase_answer
    )




    # Return the response
    return jsonify({"answer": chain.invoke({"question": question})})

# Run the Flask app on Colab
if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(' * Tunnel URL:', public_url)
    app.run(port=5000)


 * Tunnel URL: https://255e-34-86-108-162.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:51:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:51:43] "GET /favicon.ico HTTP/1.1" 404 -


SELECT `Product Description`, `Revenue` 
FROM LastWeekTopProducts 
ORDER BY `Revenue` DESC 
LIMIT 1


INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:51:50] "POST /run_query HTTP/1.1" 200 -


SELECT `Customer Name`, `Revenue` 
FROM `LastWeekTopCustomers` 
ORDER BY `Revenue` DESC 
LIMIT 10;
```


...

INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:56:16] "POST /run_query HTTP/1.1" 200 -


SELECT `Revenue` 
FROM `LastWeekSalesByDay` 
WHERE `Date` = DATE_SUB(CURDATE(), INTERVAL 1 DAY) 
	OR `Date` = DATE_SUB(CURDATE(), INTERVAL 6 DAY)
ORDER BY `Date` DESC
LIMIT 2;


INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:57:12] "POST /run_query HTTP/1.1" 200 -


SELECT AVG(`Amount`) AS `Average Order Value` FROM `LastWeekOrderStatistics` WHERE `Channel` = 'Online Retailers'


INFO:werkzeug:127.0.0.1 - - [21/Apr/2024 09:57:45] "POST /run_query HTTP/1.1" 200 -
